In [1]:
import pandas as pd
import json
import os

In [2]:
base_path = '../source/'
# Load the datasets
visitors_df = pd.read_csv(os.path.join(base_path, 'visitors.csv'))
answers_df = pd.read_csv(os.path.join(base_path, 'visitors_answers.csv'))
questions_df = pd.read_csv(os.path.join(base_path, 'visitors_questions.csv'))
exhibitor_categories_df = pd.read_csv(os.path.join(base_path, 'exhibitor_categories.csv'))
exhibitors_df = pd.read_csv(os.path.join(base_path, 'exhibitors.csv'))

## --- Processing Visitor Data ---

In [3]:
visitors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   100 non-null    object
 1   gender  100 non-null    object
 2   id      100 non-null    object
 3   data    100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [4]:
visitors_df.head()

,email,gender,id,data
0,emilija+100_L8gA@bss.mk,F,67b70a9f2d21f543a1096602,"[{ ""stepId"" : ""5c8a78336d41a10da4f730ef"", ""que..."
1,aleksandar.dimkov+mitt1_n5eA@bss.com.mk,M,67ada1ee197e604dd2722d1b,"[{ ""stepId"" : ""5c8a78336d41a10da4f730ef"", ""que..."
2,daniela.p+150_YhiF@bss.com.mk,F,67b44fef197e604dd2722d3d,"[{ ""stepId"" : ""5c8a78336d41a10da4f730ef"", ""que..."
3,tanja+182_jiPa@bss.com.mk,F,67b45018197e604dd2722d3e,"[{ ""stepId"" : ""5c8a78336d41a10da4f730ef"", ""que..."
4,aleksandar.dimkov+mitt10_V0iB@bss.com.mk,M,67b5f1392d21f543a10965f1,"[{ ""stepId"" : ""5c8a78336d41a10da4f730ef"", ""que..."


In [5]:
answers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          43 non-null     object
 1   answer      43 non-null     object
 2   questionId  43 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB


In [6]:
answers_df.head()

,id,answer,questionId
0,5c8a78336d41a10da4f73103,Personal interest,5c8a78336d41a10da4f730fe
1,5c8a78336d41a10da4f73100,To obtain general information,5c8a78336d41a10da4f730fe
2,5c8a78336d41a10da4f730ff,To source products and services,5c8a78336d41a10da4f730fe
3,5c8a78336d41a10da4f73101,To promote products and services,5c8a78336d41a10da4f730fe
4,5c8a78336d41a10da4f73102,Educational purposes,5c8a78336d41a10da4f730fe


In [7]:
print(f"Missing answers: {answers_df['answer'].isnull().sum()}")

Missing answers: 0


In [8]:
questions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5 non-null      object
 1   questionTypeId  5 non-null      object
 2   stepId          5 non-null      object
 3   question        5 non-null      object
dtypes: object(4)
memory usage: 292.0+ bytes


In [9]:
#Process the visitors data
# Initialize an empty list to hold the processed visitor data
processed_visitor_data = []

print("Processing visitor answers...")
for index, row in visitors_df.iterrows():
    visitor_id = row['id']
    visitor_email = row['email']
    visitor_gender = row['gender'] # Keep gender info

    try:
        # Load the JSON string into a Python list
        answers_list = json.loads(row['data'])

        # Iterate through each answer dictionary in the list
        for answer_data in answers_list:
            processed_visitor_data.append({
                'visitor_id': visitor_id,
                'visitor_email': visitor_email,
                'visitor_gender': visitor_gender,
                'questionId': answer_data.get('questionId'),
                'answerId': answer_data.get('answerId'), # May be empty/null if free text
                'answerValue': answer_data.get('answerValue'), # Captures free text or boolean answers
                'answerTypeId': answer_data.get('answerTypeId')
                # Add other relevant fields from answer_data if needed (like stepId)
            })
    except (json.JSONDecodeError, TypeError) as e:
        print(f"Error processing row index {index}, visitor_id {visitor_id}: {e}")
        # Decide how to handle errors: skip row, log it, etc.
        # Add a placeholder or skip if critical data is missing
        processed_visitor_data.append({
            'visitor_id': visitor_id,
             'visitor_email': visitor_email,
             'visitor_gender': visitor_gender,
             'questionId': None,
             'answerId': None,
             'answerValue': f"PARSE_ERROR: {e}",
             'answerTypeId': None
        })


# Create a new DataFrame from the processed data
visitor_answers_df = pd.DataFrame(processed_visitor_data)

visitor_answers_df

Processing visitor answers...


,visitor_id,visitor_email,visitor_gender,questionId,answerId,answerValue,answerTypeId
0,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f730f0,5c8a78336d41a10da4f730f2,,Answer
1,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,6507de7e9e117216630ef7f2,,emilija+100_L8gA@bss.mk,Answer
2,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,6507e0219e117216630ef7f5,,3138,Answer
3,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f733d8,,Eme,Answer
4,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f733e2,,Jankova,Answer
...,...,...,...,...,...,...,...
2822,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f73358,5c8a78336d41a10da4f73359,,Answer
2823,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f732ed,5c8a78336d41a10da4f732ee,,Answer
2824,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f7336d,5c8a78336d41a10da4f73370,,Answer
2825,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f7337b,5c8a78336d41a10da4f7337c,,Answer


In [10]:
visitor_answers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2827 entries, 0 to 2826
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   visitor_id      2827 non-null   object
 1   visitor_email   2827 non-null   object
 2   visitor_gender  2827 non-null   object
 3   questionId      2827 non-null   object
 4   answerId        2827 non-null   object
 5   answerValue     2827 non-null   object
 6   answerTypeId    2827 non-null   object
dtypes: object(7)
memory usage: 154.7+ KB


In [11]:
print(f"Unique visitors in processed data: {visitor_answers_df['visitor_id'].nunique()}")

Unique visitors in processed data: 100


In [12]:
# Rename columns for clarity before merging
answers_df.rename(columns={'id': 'answerId_lookup', 'answer': 'answerText', 'questionId': 'questionId_from_answer'},inplace=True)
questions_df.rename(columns={'id': 'questionId_lookup', 'question': 'questionText'},inplace=True)

In [13]:
# Merge with questions
visitor_answers_merged_df = pd.merge(
    visitor_answers_df,
    questions_df[['questionId_lookup', 'questionText']],
    left_on='questionId',
    right_on='questionId_lookup',
    how='left'
)

visitor_answers_merged_df

,visitor_id,visitor_email,visitor_gender,questionId,answerId,answerValue,answerTypeId,questionId_lookup,questionText
0,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f730f0,5c8a78336d41a10da4f730f2,,Answer,NaN,NaN
1,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,6507de7e9e117216630ef7f2,,emilija+100_L8gA@bss.mk,Answer,NaN,NaN
2,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,6507e0219e117216630ef7f5,,3138,Answer,NaN,NaN
3,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f733d8,,Eme,Answer,NaN,NaN
4,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f733e2,,Jankova,Answer,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2822,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f73358,5c8a78336d41a10da4f73359,,Answer,NaN,NaN
2823,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f732ed,5c8a78336d41a10da4f732ee,,Answer,NaN,NaN
2824,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f7336d,5c8a78336d41a10da4f73370,,Answer,5c8a78336d41a10da4f7336d,What role do you play in the purchasing decisi...
2825,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f7337b,5c8a78336d41a10da4f7337c,,Answer,NaN,NaN


In [14]:
# Merge with answers - Use the answerId for mapping predefined answers
visitor_answers_merged_df = pd.merge(
    visitor_answers_merged_df,
    answers_df[['answerId_lookup', 'answerText']],
    left_on='answerId',
    right_on='answerId_lookup',
    how='left'
)

visitor_answers_merged_df

,visitor_id,visitor_email,visitor_gender,questionId,answerId,answerValue,answerTypeId,questionId_lookup,questionText,answerId_lookup,answerText
0,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f730f0,5c8a78336d41a10da4f730f2,,Answer,NaN,NaN,NaN,NaN
1,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,6507de7e9e117216630ef7f2,,emilija+100_L8gA@bss.mk,Answer,NaN,NaN,NaN,NaN
2,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,6507e0219e117216630ef7f5,,3138,Answer,NaN,NaN,NaN,NaN
3,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f733d8,,Eme,Answer,NaN,NaN,NaN,NaN
4,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f733e2,,Jankova,Answer,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2822,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f73358,5c8a78336d41a10da4f73359,,Answer,NaN,NaN,NaN,NaN
2823,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f732ed,5c8a78336d41a10da4f732ee,,Answer,NaN,NaN,NaN,NaN
2824,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f7336d,5c8a78336d41a10da4f73370,,Answer,5c8a78336d41a10da4f7336d,What role do you play in the purchasing decisi...,5c8a78336d41a10da4f73370,Advisory role
2825,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f7337b,5c8a78336d41a10da4f7337c,,Answer,NaN,NaN,NaN,NaN


In [15]:
# Removing rows where questionId_lookup	questionTex answerId_lookup	answerText are null
visitor_answers_merged_df.dropna(subset=['questionId_lookup', 'questionText', 'answerId_lookup', 'answerText'], inplace=True)
# Reset index after dropping rows
visitor_answers_merged_df.reset_index(drop=True, inplace=True)
visitor_answers_merged_df

,visitor_id,visitor_email,visitor_gender,questionId,answerId,answerValue,answerTypeId,questionId_lookup,questionText,answerId_lookup,answerText
0,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f730fe,5c8a78336d41a10da4f73100,,Answer,5c8a78336d41a10da4f730fe,Reason for Attending the Event,5c8a78336d41a10da4f73100,To obtain general information
1,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f73227,5c8a78336d41a10da4f73244,,Answer,5c8a78336d41a10da4f73227,Which of the following best describes your job...,5c8a78336d41a10da4f73244,Media
2,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f73253,5c8a78336d41a10da4f73291,,Answer,5c8a78336d41a10da4f73253,Please indicate your company's main area of bu...,5c8a78336d41a10da4f73291,Travel Agent
3,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f7336d,5c8a78336d41a10da4f73371,,Answer,5c8a78336d41a10da4f7336d,What role do you play in the purchasing decisi...,5c8a78336d41a10da4f73371,No influence
4,67ada1ee197e604dd2722d1b,aleksandar.dimkov+mitt1_n5eA@bss.com.mk,M,5c8a78336d41a10da4f730fe,5c8a78336d41a10da4f730ff,,Answer,5c8a78336d41a10da4f730fe,Reason for Attending the Event,5c8a78336d41a10da4f730ff,To source products and services
...,...,...,...,...,...,...,...,...,...,...,...
356,uud1nluegdqmisnfzpg75iqg,daniela.p+200_IzcX_QOfF@bss.com.mk,M,5c8a78336d41a10da4f7336d,5c8a78336d41a10da4f73370,,Answer,5c8a78336d41a10da4f7336d,What role do you play in the purchasing decisi...,5c8a78336d41a10da4f73370,Advisory role
357,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f730fe,5c8a78336d41a10da4f73100,,Answer,5c8a78336d41a10da4f730fe,Reason for Attending the Event,5c8a78336d41a10da4f73100,To obtain general information
358,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f73227,5c8a78336d41a10da4f7323d,,Answer,5c8a78336d41a10da4f73227,Which of the following best describes your job...,5c8a78336d41a10da4f7323d,Guided tour services
359,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f73253,5c8a78336d41a10da4f73291,,Answer,5c8a78336d41a10da4f73253,Please indicate your company's main area of bu...,5c8a78336d41a10da4f73291,Travel Agent


In [16]:
# Drop redundant columns
visitor_answers_merged_df = visitor_answers_merged_df.drop(columns=['questionId_lookup', 'answerId_lookup','answerValue'])
visitor_answers_merged_df

,visitor_id,visitor_email,visitor_gender,questionId,answerId,answerTypeId,questionText,answerText
0,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f730fe,5c8a78336d41a10da4f73100,Answer,Reason for Attending the Event,To obtain general information
1,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f73227,5c8a78336d41a10da4f73244,Answer,Which of the following best describes your job...,Media
2,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f73253,5c8a78336d41a10da4f73291,Answer,Please indicate your company's main area of bu...,Travel Agent
3,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f7336d,5c8a78336d41a10da4f73371,Answer,What role do you play in the purchasing decisi...,No influence
4,67ada1ee197e604dd2722d1b,aleksandar.dimkov+mitt1_n5eA@bss.com.mk,M,5c8a78336d41a10da4f730fe,5c8a78336d41a10da4f730ff,Answer,Reason for Attending the Event,To source products and services
...,...,...,...,...,...,...,...,...
356,uud1nluegdqmisnfzpg75iqg,daniela.p+200_IzcX_QOfF@bss.com.mk,M,5c8a78336d41a10da4f7336d,5c8a78336d41a10da4f73370,Answer,What role do you play in the purchasing decisi...,Advisory role
357,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f730fe,5c8a78336d41a10da4f73100,Answer,Reason for Attending the Event,To obtain general information
358,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f73227,5c8a78336d41a10da4f7323d,Answer,Which of the following best describes your job...,Guided tour services
359,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f73253,5c8a78336d41a10da4f73291,Answer,Please indicate your company's main area of bu...,Travel Agent


In [17]:
#Arrange the columns in a more logical order and save it in a final df
final_visitor_data = visitor_answers_merged_df[['visitor_id','visitor_email','visitor_gender','questionId','questionText','answerId','answerText','answerTypeId']].copy()
final_visitor_data

,visitor_id,visitor_email,visitor_gender,questionId,questionText,answerId,answerText,answerTypeId
0,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f730fe,Reason for Attending the Event,5c8a78336d41a10da4f73100,To obtain general information,Answer
1,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f73227,Which of the following best describes your job...,5c8a78336d41a10da4f73244,Media,Answer
2,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f73253,Please indicate your company's main area of bu...,5c8a78336d41a10da4f73291,Travel Agent,Answer
3,67b70a9f2d21f543a1096602,emilija+100_L8gA@bss.mk,F,5c8a78336d41a10da4f7336d,What role do you play in the purchasing decisi...,5c8a78336d41a10da4f73371,No influence,Answer
4,67ada1ee197e604dd2722d1b,aleksandar.dimkov+mitt1_n5eA@bss.com.mk,M,5c8a78336d41a10da4f730fe,Reason for Attending the Event,5c8a78336d41a10da4f730ff,To source products and services,Answer
...,...,...,...,...,...,...,...,...
356,uud1nluegdqmisnfzpg75iqg,daniela.p+200_IzcX_QOfF@bss.com.mk,M,5c8a78336d41a10da4f7336d,What role do you play in the purchasing decisi...,5c8a78336d41a10da4f73370,Advisory role,Answer
357,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f730fe,Reason for Attending the Event,5c8a78336d41a10da4f73100,To obtain general information,Answer
358,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f73227,Which of the following best describes your job...,5c8a78336d41a10da4f7323d,Guided tour services,Answer
359,mv0j9yovwhgmtif9l1naciko,daniela.p+200_IzcX_CIHU@bss.com.mk,M,5c8a78336d41a10da4f73253,Please indicate your company's main area of bu...,5c8a78336d41a10da4f73291,Travel Agent,Answer


## --- Processing Exhibitor Data ---

In [18]:
exhibitors_df.head()

,exhibitorid,Name,MainCategories
0,90556,Turkey Travels,52276|52280|52281
1,92462,Russian Travel Company,52273|52283|52289|52291|52298|52302
2,92491,Indian Travel Company,52273|52274|52281
3,92492,Asia Tourism,52272|52276|52352|52358
4,92493,SriLanka Adventures,52296|52327|52352


In [19]:
# Split the pipe-separated string into a list of category IDs
exhibitors_df['CategoryList'] = exhibitors_df['MainCategories'].apply(lambda x: x.split('|') if x else [])

In [20]:
exhibitors_df.head()

,exhibitorid,Name,MainCategories,CategoryList
0,90556,Turkey Travels,52276|52280|52281,"[52276, 52280, 52281]"
1,92462,Russian Travel Company,52273|52283|52289|52291|52298|52302,"[52273, 52283, 52289, 52291, 52298, 52302]"
2,92491,Indian Travel Company,52273|52274|52281,"[52273, 52274, 52281]"
3,92492,Asia Tourism,52272|52276|52352|52358,"[52272, 52276, 52352, 52358]"
4,92493,SriLanka Adventures,52296|52327|52352,"[52296, 52327, 52352]"


In [21]:
# Check if any empty category lists exist
exhibitors_df[exhibitors_df['CategoryList']=='[]']

,exhibitorid,Name,MainCategories,CategoryList


In [22]:
# Explode the DataFrame to have one row per exhibitor-category pair
exhibitors_exploded_df = exhibitors_df.explode('CategoryList')
# Rename the column holding the single category ID
exhibitors_exploded_df.rename(columns={'CategoryList': 'categoryId'},inplace=True)

exhibitors_exploded_df.head()

,exhibitorid,Name,MainCategories,categoryId
0,90556,Turkey Travels,52276|52280|52281,52276
0,90556,Turkey Travels,52276|52280|52281,52280
0,90556,Turkey Travels,52276|52280|52281,52281
1,92462,Russian Travel Company,52273|52283|52289|52291|52298|52302,52273
1,92462,Russian Travel Company,52273|52283|52289|52291|52298|52302,52283


In [23]:
exhibitor_categories_df.head()

,categoryId,categoryName
0,52271,1. Accomodation providers
1,52272,1.1 Hotel / Hotel chain / Inn
2,52273,1.2 Apartments / Residential hotel
3,52274,1.3 Hostel / Motel
4,52275,1.4 Boarding house


In [24]:
exhibitor_categories_df

,categoryId,categoryName
0,52271,1. Accomodation providers
1,52272,1.1 Hotel / Hotel chain / Inn
2,52273,1.2 Apartments / Residential hotel
3,52274,1.3 Hostel / Motel
4,52275,1.4 Boarding house
...,...,...
91,52319,9.1 Hotel designer
92,52320,9.2 Manufacturer / distributor of consumables
93,52321,9.3 Cleaning
94,52322,9.4 Security system manufacturer / distributor


In [25]:
# Get the parent category for each exhibitor based on the category name
def get_parent_category(row):
    category_name = row["categoryName"]
    
    if "." in category_name:  # Check if it's a subcategory
        parent_num = category_name.split(".")[0]  # Extract the main category number
        parent_row = exhibitor_categories_df[exhibitor_categories_df["categoryName"].str.match(f"^{parent_num}\D")].iloc[0]  # Find the parent category row
        return parent_row["categoryName"]
    
    return category_name  # If it's a parent category, keep it the same

exhibitor_categories_df["parentCategory"] = exhibitor_categories_df.apply(get_parent_category, axis=1)

exhibitor_categories_df

,categoryId,categoryName,parentCategory
0,52271,1. Accomodation providers,1. Accomodation providers
1,52272,1.1 Hotel / Hotel chain / Inn,1. Accomodation providers
2,52273,1.2 Apartments / Residential hotel,1. Accomodation providers
3,52274,1.3 Hostel / Motel,1. Accomodation providers
4,52275,1.4 Boarding house,1. Accomodation providers
...,...,...,...
91,52319,9.1 Hotel designer,9. Hotel equipment & service suppliers
92,52320,9.2 Manufacturer / distributor of consumables,9. Hotel equipment & service suppliers
93,52321,9.3 Cleaning,9. Hotel equipment & service suppliers
94,52322,9.4 Security system manufacturer / distributor,9. Hotel equipment & service suppliers


In [26]:
# Ensure correct data type for merging
# Convert both categoryId columns to a common type (e.g., string or int)
# Check dtypes first! Let's assume they should be integers based on exhibitor_categories.csv
exhibitors_exploded_df['categoryId'] = pd.to_numeric(exhibitors_exploded_df['categoryId'], errors='coerce')
exhibitor_categories_df['categoryId'] = pd.to_numeric(exhibitor_categories_df['categoryId'], errors='coerce')

print(exhibitors_exploded_df['categoryId'].isnull().sum(), exhibitor_categories_df['categoryId'].isnull().sum())

0 0


In [27]:
exhibitors_exploded_df['categoryId'] = exhibitors_exploded_df['categoryId'].astype(int)
exhibitor_categories_df['categoryId'] = exhibitor_categories_df['categoryId'].astype(int)

In [28]:
exhibitors_final_df = pd.merge(
    exhibitors_exploded_df,
    exhibitor_categories_df,
    on='categoryId',
    how='left'
)
exhibitors_final_df

,exhibitorid,Name,MainCategories,categoryId,categoryName,parentCategory
0,90556,Turkey Travels,52276|52280|52281,52276,1.5 Resort hotel,1. Accomodation providers
1,90556,Turkey Travels,52276|52280|52281,52280,2.1 Inbound tour operator,2. Tour operators
2,90556,Turkey Travels,52276|52280|52281,52281,2.2 Outbound tour operator,2. Tour operators
3,92462,Russian Travel Company,52273|52283|52289|52291|52298|52302,52273,1.2 Apartments / Residential hotel,1. Accomodation providers
4,92462,Russian Travel Company,52273|52283|52289|52291|52298|52302,52283,2.4 Mass market tour operators,2. Tour operators
...,...,...,...,...,...,...
178,17729,Sunny Travel Journeys,52271|52359|52304|52285|52282|52272|52329|5236...,52282,2.3 Receptive / ground operator,2. Tour operators
179,17729,Sunny Travel Journeys,52271|52359|52304|52285|52282|52272|52329|5236...,52272,1.1 Hotel / Hotel chain / Inn,1. Accomodation providers
180,17729,Sunny Travel Journeys,52271|52359|52304|52285|52282|52272|52329|5236...,52329,10.5 Zoo,10. Museums & parks
181,17729,Sunny Travel Journeys,52271|52359|52304|52285|52282|52272|52329|5236...,52366,"17.3 Banking, investments",17. Insurance companies & banking


In [29]:
exhibitors_final_df = exhibitors_final_df[['exhibitorid', 'Name', 'categoryId', 'categoryName','parentCategory']].copy()
exhibitors_final_df.rename(columns={'Name': 'exhibitorName'}, inplace=True)
exhibitors_final_df

,exhibitorid,exhibitorName,categoryId,categoryName,parentCategory
0,90556,Turkey Travels,52276,1.5 Resort hotel,1. Accomodation providers
1,90556,Turkey Travels,52280,2.1 Inbound tour operator,2. Tour operators
2,90556,Turkey Travels,52281,2.2 Outbound tour operator,2. Tour operators
3,92462,Russian Travel Company,52273,1.2 Apartments / Residential hotel,1. Accomodation providers
4,92462,Russian Travel Company,52283,2.4 Mass market tour operators,2. Tour operators
...,...,...,...,...,...
178,17729,Sunny Travel Journeys,52282,2.3 Receptive / ground operator,2. Tour operators
179,17729,Sunny Travel Journeys,52272,1.1 Hotel / Hotel chain / Inn,1. Accomodation providers
180,17729,Sunny Travel Journeys,52329,10.5 Zoo,10. Museums & parks
181,17729,Sunny Travel Journeys,52366,"17.3 Banking, investments",17. Insurance companies & banking


In [30]:
#Save Processed Data
final_visitor_data.to_csv(os.path.join(base_path, 'processed_visitors_answers.csv'), index=False)
exhibitors_final_df.to_csv(os.path.join(base_path, 'processed_exhibitors_categories.csv'), index=False)